### Projecy

##### Import Packages

In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
pd.set_option("display.max_columns",30)

#### Import Datas

In [110]:
df = pd.read_csv("/Users/carloalbertoguaitoli/BankMarketing/BankMarketing.csv", sep=";")
print("We have", df.shape[0], "records with", df.shape[1],"vars")
df.head()

We have 9280 records with 21 vars


,ID,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,34579,35,admin.,single,university.degree,no,yes,no,cellular,may,thu,1,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
1,446,42,technician,married,professional.course,no,no,no,telephone,may,tue,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
2,20173,36,admin.,married,university.degree,no,no,no,cellular,aug,mon,2,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,yes
3,18171,37,admin.,married,high.school,no,yes,yes,telephone,jul,wed,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,yes
4,30128,31,management,single,university.degree,no,yes,no,cellular,apr,thu,1,999,0,nonexistent,-1.8,93.075,-47.1,1.365,5099.1,no


#### Define the objective

Our target is to predict if a customer will subscribe a banking tool (term deposit).

According to our target, we define the statistical unit as "the single customer defined by the single ID".

##### Clean Datas

In [111]:
df.describe()

,ID,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000
mean,24826.214224,40.458836,2.320905,888.347198,0.316703,-0.492748,93.482002,-40.181897,2.968123,5135.510938
std,12425.733760,11.963006,2.294474,312.500496,0.699808,1.723356,0.634630,5.359290,1.890823,87.040463
min,1.000000,17.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,14310.500000,32.000000,1.000000,999.000000,0.000000,-1.800000,92.893000,-42.700000,1.244000,5076.200000
50%,27390.000000,38.000000,2.000000,999.000000,0.000000,-0.100000,93.444000,-41.800000,4.021000,5191.000000
75%,36751.500000,48.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.959000,5228.100000
max,41186.000000,98.000000,35.000000,999.000000,6.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


#### Check categorical Variables

In [148]:
#Controllare distribuzione età
df.job.value_counts() # Lasciamo / Moda #Chiediamo #0.80
df.marital.value_counts() #Moda (FATTO)
#df.education.value_counts() #CrossTab e #Unknown Moda 
#df.default.value_counts() # Eliminare # Proviamo a vedere di lasciare unknown /
#df.housing.value_counts() #Moda /  Ma valutiamo
#df.loan.value_counts() #Moda (FATTO)
df.month.value_counts() #Featuring engeneering
#df.day_of_week.value_counts()
#df.poutcome.value_counts() # Lasciare così

lol    2512
jul    1462
aug    1348
jun    1187
nov     880
apr     813
oct     377
mar     308
sep     294
dec      99
Name: month, dtype: int64

In [132]:
df["loan"].replace({'unknown': "no"}, inplace=True)
df["marital"].replace({'unknown': "married"}, inplace=True)
df.head()

,ID,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,ciao,35,admin.,single,university.degree,no,yes,no,cellular,may,thu,1,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
1,446,42,technician,married,professional.course,no,no,no,telephone,may,tue,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
2,20173,36,admin.,married,university.degree,no,no,no,cellular,aug,mon,2,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,yes
3,18171,37,admin.,married,high.school,no,yes,yes,telephone,jul,wed,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,yes
4,30128,31,management,single,university.degree,no,yes,no,cellular,apr,thu,1,999,0,nonexistent,-1.8,93.075,-47.1,1.365,5099.1,no


Let's check for any Nas

In [134]:
df = df.drop("ID", axis=1)
df = df.drop("contact", axis=1)
df.head()

,age,job,marital,education,default,housing,loan,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,35,admin.,single,university.degree,no,yes,no,may,thu,1,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
1,42,technician,married,professional.course,no,no,no,may,tue,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
2,36,admin.,married,university.degree,no,no,no,aug,mon,2,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,yes
3,37,admin.,married,high.school,no,yes,yes,jul,wed,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,yes
4,31,management,single,university.degree,no,yes,no,apr,thu,1,999,0,nonexistent,-1.8,93.075,-47.1,1.365,5099.1,no


In [147]:
df["month"].replace({'mar': 3,"feb":2,"march"}, inplace=True)



/Users/carloalbertoguaitoli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


,age,job,marital,education,default,housing,loan,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,35,admin.,single,university.degree,no,yes,no,lol,thu,1,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
1,42,technician,married,professional.course,no,no,no,lol,tue,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
2,36,admin.,married,university.degree,no,no,no,aug,mon,2,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,yes
3,37,admin.,married,high.school,no,yes,yes,jul,wed,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,yes
4,31,management,single,university.degree,no,yes,no,apr,thu,1,999,0,nonexistent,-1.8,93.075,-47.1,1.365,5099.1,no


TypeError: function missing required argument 'year' (pos 1)